In [0]:
!pip install scikit-multilearn

     |████████████████████████████████| 92kB 2.8MB/s 


In [0]:
cd drive/My\ Drive/1003\ Project

/content/drive/My Drive/1003 Project


# Load data

In [0]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import label_ranking_average_precision_score as LRAP

In [0]:
def load_data(file_name):

    # load data from csv
    data = pd.read_csv(file_name, usecols=['labels', 'features'])

    # remove rows without proper label
    rows_to_remove = [i for i in range(len(data)) if ':' in data.loc[i,'labels']]
    data.drop(rows_to_remove, inplace=True)
    data.reset_index(drop=True, inplace=True)

    # extract features from sparse representation
    feature = np.zeros((len(data), 5000))
    for i in range(len(data)):
        for j in data.loc[i,'features'].replace('\n','').split():
            ft, val = j.split(':')
            feature[i,int(ft)] = float(val)
    X = pd.DataFrame(feature)

    # extract labels
    y = data['labels'].map(lambda x: tuple([int(i) for i in x.replace(' ','').split(',')]))
    
    return X, y

In [0]:
X_train, y_train = load_data("train.csv")
X_val, y_val = load_data('dev.csv')

In [0]:
X_train.shape

(15511, 5000)

In [0]:
binarizer = MultiLabelBinarizer()
binary_y_train = binarizer.fit_transform(y_train)

In [0]:
binary_y_train.shape

(15511, 3786)

## Classifier Chain with RF

In [0]:
from skmultilearn.problem_transform import ClassifierChain
from sklearn.ensemble import RandomForestClassifier
import time
start_time = time.time()
clf = ClassifierChain(RandomForestClassifier())
clf.fit(X_train, binary_y_train)
end_time = time.time()

In [0]:
print('Training time:', end_time - start_time)

Training time: 11963.349047899246


In [0]:
binary_y_val = binarizer.transform(y_val)
y_val_pred = clf.predict_proba(X_val)
print('LRAP: ', LRAP(binary_y_val, y_val_pred.toarray()))

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:987: UserWarning: unknown class(es) [1171, 1377, 1717, 1724, 1861, 1918, 1936, 2107, 2119, 2122, 2170, 2202, 2209, 2227, 2249, 2267, 2275, 2311, 2312, 2417, 2471, 2515, 2538, 2560, 2576, 2595, 2617, 2626, 2632, 2653, 2654, 2656, 2672, 2736, 2770, 2771, 2781, 2823, 2858, 2936, 2937, 2945, 2946, 2999, 3027, 3078, 3094, 3114, 3126, 3128, 3131, 3169, 3193, 3205, 3218, 3264, 3309, 3343, 3348, 3358, 3366, 3373, 3383, 3401, 3415, 3468, 3471, 3480, 3481, 3482, 3522, 3523, 3530, 3544, 3546, 3587, 3598, 3607, 3654, 3677, 3678, 3681, 3687, 3701, 3711, 3717, 3723, 3724, 3725, 3727, 3728, 3729, 3749, 3752, 3755, 3771, 3832, 3853, 3882, 3885, 3894, 3898, 3903, 3915, 3954, 3969, 3971, 3972, 466, 469, 788] will be ignored
  .format(sorted(unknown, key=str)))


LRAP:  0.5551620623956518


In [0]:
from joblib import dump, load
dump(clf, 'CC_RF.joblib') 

['CC_RF.joblib']

In [0]:
# clf = load('CC_RF.joblib') 

In [0]:
# binary_y_val = binarizer.transform(y_val)
# y_val_pred = clf.predict_proba(X_val)
# print('LRAP: ', LRAP(binary_y_val, y_val_pred.toarray()))

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:987: UserWarning: unknown class(es) [1171, 1377, 1717, 1724, 1861, 1918, 1936, 2107, 2119, 2122, 2170, 2202, 2209, 2227, 2249, 2267, 2275, 2311, 2312, 2417, 2471, 2515, 2538, 2560, 2576, 2595, 2617, 2626, 2632, 2653, 2654, 2656, 2672, 2736, 2770, 2771, 2781, 2823, 2858, 2936, 2937, 2945, 2946, 2999, 3027, 3078, 3094, 3114, 3126, 3128, 3131, 3169, 3193, 3205, 3218, 3264, 3309, 3343, 3348, 3358, 3366, 3373, 3383, 3401, 3415, 3468, 3471, 3480, 3481, 3482, 3522, 3523, 3530, 3544, 3546, 3587, 3598, 3607, 3654, 3677, 3678, 3681, 3687, 3701, 3711, 3717, 3723, 3724, 3725, 3727, 3728, 3729, 3749, 3752, 3755, 3771, 3832, 3853, 3882, 3885, 3894, 3898, 3903, 3915, 3954, 3969, 3971, 3972, 466, 469, 788] will be ignored
  .format(sorted(unknown, key=str)))


LRAP:  0.5551620623956518
